In [0]:
# This Downloads this notebook directly to a Google Drive Storage for GUI based information retrieval.

from google.colab import drive
drive.mount('/content/drive')

In [0]:
# Here we import the necessary libraries
# the Google library to create neural network graphs 
import tensorflow as tf
import numpy as np
import pandas as pd
import time
import os

In [24]:
# cpu benchmark:

def add_op(x,y):
  return(x+y)
x = tf.placeholder(tf.float32,[1000000,10])
y = tf.placeholder(tf.float32,[1000000,10])
session = tf.Session()

with tf.Session() as session:
  start = time.time()
  result = session.run(add_op(x,y), feed_dict = {x: np.zeros([1000000,10]), y: np.zeros([1000000,10])})
  end = time.time()
  print(end-start)

0.23328375816345215


In [25]:

def multi_op(x,y):
  return(x*y)
x = tf.placeholder(tf.float32,[1000000,10])
y = tf.placeholder(tf.float32,[1000000,10])
session = tf.Session()

with tf.Session() as session:
  start = time.time()
  result = session.run(multi_op(x,y), feed_dict = {x: np.zeros([1000000,10]), y: np.zeros([1000000,10])})
  end = time.time()
  print(end-start)

0.1917870044708252


Here we see the evaluation time for the addition operation on an 8 dimensional array takes about a two-hundreths of a second with a k80 gpu. Now let's change the run-time from GPU to TPU



In [13]:

tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is ', tpu_address)

with tf.Session(tpu_address) as session:
  devices = session.list_devices()
print("TPU devices: ", devices)

TPU address is  grpc://10.84.36.210:8470
TPU devices:  [_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 6439944083457219433), _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 11962535389993995849), _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 892862032677600460), _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 16493255044049265489), _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 13265266298638696549), _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 9723401090128625952), _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 5622812162628437592), _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 16632792574135499589), _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 13192522060

In [14]:
def add_op(x,y):
  return x+y

x = tf.placeholder(tf.float32,[1000000,10])
y = tf.placeholder(tf.float32,[1000000,10])

tpu_ops = tf.contrib.tpu.rewrite(add_op,[x,y])
session = tf.Session(tpu_address)

try:
  session.run(tf.contrib.tpu.initialize_system())
  start = time.time()
  print(session.run(tpu_ops, feed_dict = {x: np.zeros([1000000,10]), y: np.zeros([1000000,10])}))
  end = time.time()
  print(end-start)
  
finally:
  session.run(tf.contrib.tpu.shutdown_system(),)
  session.close()

[array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)]
0.9194293022155762


In [15]:
def multi_op(x,y):
  return x*y

x = tf.placeholder(tf.float32,[1000000,10])
y = tf.placeholder(tf.float32,[1000000,10])

tpu_ops = tf.contrib.tpu.rewrite(multi_op,[x,y])
session = tf.Session(tpu_address)

try:
  session.run(tf.contrib.tpu.initialize_system())
  start = time.time()
  print(session.run(tpu_ops, feed_dict = {x: np.zeros([1000000,10]), y: np.zeros([1000000,10])}))
  end = time.time()
  print(end-start)
  
finally:
  session.run(tf.contrib.tpu.shutdown_system(),)
  session.close()

[array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)]
0.5458250045776367


In [10]:
x.shape

(300, 5)

We see a noticeable speed-up with the multiply operation, still a little sluggish when compared to the k80 gpu.



In [16]:
!echo "# TPU_GPU_CPU_comparison_with_Tensor_Operations" >> README.md
!git init
!git add README.md
!git commit -m "first commit"
!git remote add origin https://github.com/mark-t-conrad/TPU_GPU_CPU_comparison_with_Tensor_Operations.git
!git push -u origin master

Initialized empty Git repository in /content/.git/

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@3cb7b0c3baa9.(none)')
error: src refspec master does not match any.
error: failed to push some refs to 'https://github.com/mark-t-conrad/TPU_GPU_CPU_comparison_with_Tensor_Operations.git'
